In [75]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [150]:
outbound_data = pd.read_excel("Outbound.xlsx")
demand_data = pd.read_excel("Demand Forecast.xlsx")
unit_data = pd.read_excel('Product Data per State.xlsx')
handling_out_data = pd.read_excel('Warehousing.xlsx')

In [151]:
# What I did: removed the State column, remove the description of tariff and small shipments
outbound_data.index = outbound_data.State
demand_data.index = demand_data.state
outbound_data = outbound_data.drop(['State','Small shipment'], axis=1)

In [153]:
# As-Is situation

as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

dc_product_demand = {}
for dc in as_is_dc:
    dc_product_demand[dc] = {}  # Create a dictionary for each DC
    for product in ['blender', 'swing', 'chair', 'scooter', 'skiprope']:
        total_demand_per_dc = 0
        for state in as_is_dc[dc]:
            total_demand_per_dc += (demand_data.loc[state, product])
        dc_product_demand[dc][product] = total_demand_per_dc  # Store demand (in boxes) for each product based on DC

dc_product_demand

{'NY': {'blender': 226319,
  'swing': 6468,
  'chair': 161655,
  'scooter': 3233,
  'skiprope': 969935},
 'ND': {'blender': 31949,
  'swing': 913,
  'chair': 22820,
  'scooter': 22832,
  'skiprope': 136921},
 'IL': {'blender': 271812,
  'swing': 7766,
  'chair': 194150,
  'scooter': 3884,
  'skiprope': 1164904},
 'TN': {'blender': 216583,
  'swing': 6188,
  'chair': 154702,
  'scooter': 3095,
  'skiprope': 928213},
 'WA': {'blender': 51437,
  'swing': 1470,
  'chair': 36741,
  'scooter': 735,
  'skiprope': 220442},
 'TX': {'blender': 190768,
  'swing': 5451,
  'chair': 136263,
  'scooter': 2725,
  'skiprope': 817575},
 'UT': {'blender': 179502,
  'swing': 5129,
  'chair': 128215,
  'scooter': 2564,
  'skiprope': 769295}}

In [154]:
# Calculate the outbound costs
shipping_costs = []
handling_out_costs_per_dc = []
product_units = unit_data.columns.difference(['state'])
handling_out_data.index = handling_out_data.DC
unit_data.index = unit_data.state

for state in demand_data.index:
    for dc, states in as_is_dc.items():
        if state in states:
            shipping_costs.append((demand_data['total_weight'][state] * outbound_data.loc[state, dc]))
            # handling_out_costs.append(demand_data['blender_boxes'] * handling_out_data.loc[dc, ])

for dc, states in as_is_dc.items():
    out_costs_per_state = 0
    for state in unit_data.index:
        if state in states:
            for product in product_units:
                out_costs_per_state += unit_data[product][state] * handling_out_data.loc[dc, product]
            
    handling_out_costs_per_dc.append(out_costs_per_state)

shipping_costs = sum(shipping_costs)
handling_out_costs = sum(handling_out_costs_per_dc)
opening_costs = len(as_is_dc)*1000000 # Keep a DC open need $1M per year.
total_outbound_costs = np.round(shipping_costs + handling_out_costs + opening_costs, 1)

print('Total Shipping Costs (from DC to Customers):', shipping_costs)
print('Total Handling Out Costs:', handling_out_costs)
print('Total costs for keeping 7 DCs open:', opening_costs)
print('Total Outbound Costs:', total_outbound_costs)

Total Shipping Costs (from DC to Customers): 1135481.33914202
Total Handling Out Costs: 1845542.21
Total costs for keeping 7 DCs open: 7000000
Total Outbound Costs: 9981023.5


In [81]:
# Determine To-Be situation
best_dc_allocation = {}
# then check everything! (also ask why DC is not in the list)
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in best_dc_allocation:
        best_dc_allocation[best_dc] = state
    elif isinstance(best_dc_allocation[best_dc], list):
        best_dc_allocation[best_dc].append(state)
    else:
        best_dc_allocation[best_dc] = [best_dc_allocation[best_dc], state]
        
print(best_dc_allocation)

{'WA': ['AK', 'ID', 'MT', 'OR', 'WA'], 'TN': ['AL', 'FL', 'GA', 'KY', 'MS', 'NC', 'OH', 'SC', 'VA', 'WV'], 'TX': ['AR', 'LA', 'NM', 'OK', 'TN', 'TX'], 'UT': ['AZ', 'CO', 'NV', 'UT', 'WY'], 'CA': 'CA', 'NY': ['CT', 'DE', 'MA', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT'], 'PA': ['HI', 'MD', 'PA', 'DC'], 'KS': ['IA', 'KS', 'MO', 'NE'], 'IL': ['IL', 'MI', 'MN'], 'ND': ['IN', 'ND', 'SD', 'WI']}
